<a href="https://colab.research.google.com/github/mallibus/TF2---colab/blob/master/Colab_5_Building_a_Recurrent_Neural_Network_in_TensorFlow_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Installing the dependencies and setting up a GPU environment

In [3]:
!pip install tensorflow-gpu==2.0.0.alpha0

     |████████████████████████████████| 332.1MB 58kB/s 
     |████████████████████████████████| 3.0MB 44.7MB/s 
     |████████████████████████████████| 419kB 50.7MB/s 


## Step 2: Importing the libraries

In [0]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb

In [2]:
tf.__version__

'2.0.0-alpha0'

## Step 3: Data Preprocessing

### Setting up the dataset parameters

In [0]:
number_of_words = 20000
max_len = 100

### Loading the IMDB dataset

Cell added to allow imdb load - otherwise I got error  
`ValueError: Object arrays cannot be loaded when allow_pickle=False`

The fix comes from [here](https://stackoverflow.com/questions/55890813/how-to-fix-object-arrays-cannot-be-loaded-when-allow-pickle-false-for-imdb-loa)



In [4]:

import numpy as np
old = np.load
np.load = lambda *a,**k: old(*a,**k,allow_pickle=True)

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=number_of_words)

np.load = old
del(old)

17465344/17464789 [==============================] - 1s 0us/step


In [0]:
# This does not work
# (X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=number_of_words)

ValueError: ignored

## Explore some reviews

To convert back from list of numbers to words with some sense, there is some work to do. [Here is where the code comes from](https://stackoverflow.com/questions/42821330/restore-original-text-from-keras-s-imdb-dataset).

In [5]:
NUM_WORDS=number_of_words # only use top  words
INDEX_FROM=3   # word index offset

word_to_id = imdb.get_word_index()
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2

id_to_word = {value:key for key,value in word_to_id.items()}
#print(' '.join(id_to_word[id] for id in X_train[0] ))

1646592/1641221 [==============================] - 0s 0us/step


In [6]:
import textwrap
ylabels = ['Negative','Positive']
size = 3
sample = np.random.choice(range(len(X_train)),size)
for i in sample:
    print(f'Review #{i}:{ylabels[y_train[i]]}')
    s = ' '.join(id_to_word[id] for id in X_train[i])
    print('\n'.join(textwrap.wrap(s, width=120, replace_whitespace=False)))
    print()



Review #5326:Positive
<START> i realize this review will get me bashed by the expert film critics <UNK> this site but i will defend this film
br br the dentist is actually a really good film the acting isn't always top notch but the thrills are good and the
story's good plus you see linda hoffman's boobies not that i'm an expert in this field but the direction seems good and
the plot makes sense corbin makes a great creepy dentist it does to dentists what jaws does to sharks ish it obviously
had a fairly limited budget but they did well with it what they could and developed the characters well those that count
br br the end

Review #100:Negative
<START> i am a great fan of david lynch and have everything that he's made on dvd except for hotel room the 2 hour twin
peaks movie so when i found out about this i immediately grabbed it and and what is this it's a bunch of crudely drawn
black and white cartoons that are loud and foul mouthed and unfunny maybe i don't know what's good but mayb

### Padding all sequences to be the same length 

In [0]:
# Changing variable name to preserve orginal dataset
X_train_p = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_len)

In [0]:
# Changing variable name to preserve orginal dataset
X_test_p = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_len)

## Step 4: Building a Recurrent Neural Network

### Defining the model

In [0]:
model = tf.keras.Sequential()

### Adding the embedding layer

In [0]:
model.add(tf.keras.layers.Embedding(input_dim=number_of_words, output_dim=128, input_shape=(X_train_p.shape[1],)))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 128)          2560000   
Total params: 2,560,000
Trainable params: 2,560,000
Non-trainable params: 0
_________________________________________________________________


### Adding the LSTM layer

- units: 128
- activation: tanh

In [14]:
model.add(tf.keras.layers.LSTM(units=128, activation='tanh'))

W0714 16:25:02.393356 140060086777728 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f61cc82cfd0>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


### Adding the output layer

- units: 1
- activation: sigmoid

In [0]:
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

### Compiling the model

In [0]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 128)          2560000   
_________________________________________________________________
unified_lstm (UnifiedLSTM)   (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________


### Training the model

In [20]:
model.fit(X_train_p, y_train, epochs=3, batch_size=128)

Epoch 1/3
25000/25000 [==============================] - 6s 247us/sample - loss: 0.4627 - accuracy: 0.7846
Epoch 2/3
25000/25000 [==============================] - 2s 84us/sample - loss: 0.2910 - accuracy: 0.8840
Epoch 3/3
25000/25000 [==============================] - 2s 81us/sample - loss: 0.2304 - accuracy: 0.9111


### Evaluating the model

In [22]:
test_loss, test_acurracy = model.evaluate(X_test_p, y_test)

25000/25000 [==============================] - 3s 129us/sample - loss: 0.3421 - accuracy: 0.8535


In [23]:
print("Test accuracy: {}".format(test_acurracy))

Test accuracy: 0.8534799814224243


In [0]:
#model = tf.keras.Sequential()
#model.add(tf.keras.layers.Embedding(input_dim=number_of_words, output_dim=128, input_shape=(X_train_p.shape[1],)))
#model.add(tf.keras.layers.LSTM(units=128, activation='tanh'))
#model.add(tf.keras.layers.GlobalAveragePooling1D())
#model.add(tf.keras.layers.Dense(units=128, activation='relu'))
#model.add(tf.keras.layers.BatchNormalization())
#model.add(tf.keras.layers.Dropout(0.2))
#model.add(tf.keras.layers.Dense(units=128, activation='relu'))
#model.add(tf.keras.layers.BatchNormalization())
#model.add(tf.keras.layers.Dropout(0.2))
#model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
#model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
#model.fit(X_train_p, y_train, epochs=3, batch_size=128)
#test_loss, test_acurracy = model.evaluate(X_test_p, y_test)
#print("Test accuracy: {}".format(test_acurracy))

In [65]:
ylabels = ['Negative','Positive']
size = 3
sample = np.random.choice(range(len(X_test)),size)
predictions = model.predict_classes(X_test_p[sample])
for idx,i in enumerate(sample):
    print(f'Review #{i} - P:{ylabels[predictions[idx][0]]},A:{ylabels[y_test[sample[idx]]]}')
    s = ' '.join(id_to_word[id] for id in X_test[i])
    print('\n'.join(textwrap.wrap(s, width=120, replace_whitespace=False)))
    print()


Review #12853 - P:Negative,A:Negative
<START> i have just caught this movie on tcm and can understand why george murphy went into politics if this was the
best mgm could serve up to him it is so slow moving that the attempt to make it a real film noir effort does not come
off it featured two of my favourite br br players in eve arden completely wasted and dean stockwell the best actor in
the film but what really hit me was that the leading lady frances <UNK> went through some 90 minutes it seemed longer
without changing the expression on her face her <UNK> scene was comical john <UNK> played his role ok but the script let
him and the rest of the cast down very badly i gave it 4 stars mainly because of the photography it would have been on
the first half of the program when double features were the go

Review #8656 - P:Positive,A:Positive
<START> if you're in the middle of a ferocious war and it's still not clear that you're going to come out on top among
the things you'll be concerned 